# Start simulation (Do not touch)

### Import libraries

In [1]:
import numpy as np
import pybullet as p
import pybullet_data
from time import sleep

### Numpy print options (precision)

In [2]:
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.3f}".format(x)}) # print up to 3 decimal place

### Connect to GUI

In [3]:
ClientId = p.connect(p.GUI) # open client with GUI

### GUI configurations

In [4]:
p.setAdditionalSearchPath(pybullet_data.getDataPath())

p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)                        # Enable/Disable a default GUI
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 1)                    # Enable/Disable a light effect
p.configureDebugVisualizer(p.COV_ENABLE_RGB_BUFFER_PREVIEW, 0)         # Show/Hide a RGB buffer preview window
p.configureDebugVisualizer(p.COV_ENABLE_DEPTH_BUFFER_PREVIEW, 0)       # Show/Hide a Depth buffer preview window
p.configureDebugVisualizer(p.COV_ENABLE_SEGMENTATION_MARK_PREVIEW, 0)  # Show/Hide a Seg-map buffer preview window

### Simulation settings

In [5]:
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=45, cameraPitch=-30, cameraTargetPosition=[0, 0, 0.5]) # move camera view
p.setGravity(0, 0, -9.81) # set a gravity vector
p.setTimeStep(1./240)     # set a simulation time step (240 Hz)
p.setRealTimeSimulation(1) # start real-time physics simulation

### Load plane

In [6]:
PlaneId = p.loadURDF("plane.urdf") # ground

### coordinate frame visualization function

In [7]:
def visualizeEEframe(robotId, n=6):
    LineID = []
    for i in range(3): # iteration for X, Y, Z
        # draw axis in body frame
        id = p.addUserDebugLine(lineFromXYZ=[0, 0, 0.06], lineToXYZ=0.1*np.eye(3)[i] + [0, 0, 0.06], lineColorRGB=np.eye(3)[i], lineWidth=5, 
                                parentObjectUniqueId=robotId, parentLinkIndex=n-1, physicsClientId=ClientId)
        LineID.append(id)
    return LineID

def visualizeDHframe(ballId):
    LineID = []
    for i in range(3):# iteration for X, Y, Z
        # draw axis in body frame
        id = p.addUserDebugLine(lineFromXYZ=[0, 0, 0], lineToXYZ=0.1*np.eye(3)[i], lineColorRGB=np.eye(3)[i], lineWidth=5, 
                                parentObjectUniqueId=ballId, physicsClientId=ClientId)
        LineID.append(id)
    return LineID

### Load robot

In [8]:
# Load robot and read joint informations
urdf_path = "./indy7_v2/model.urdf"
base_pos = [0, 0, 0]
base_quat = p.getQuaternionFromEuler([0, 0, 0])

flags = p.URDF_USE_INERTIA_FROM_FILE + p.URDF_USE_SELF_COLLISION + p.URDF_USE_SELF_COLLISION_EXCLUDE_PARENT

robotId = p.loadURDF(urdf_path, 
                     basePosition=base_pos, 
                     baseOrientation=base_quat,
                     flags=flags, 
                     physicsClientId=ClientId)
numJoints = p.getNumJoints(robotId, physicsClientId=ClientId)
jointTypes = [p.getJointInfo(robotId, i)[2] for i in range(numJoints)]
MovableJointIdx = [i for i, jointType in enumerate(jointTypes) if jointType==p.JOINT_REVOLUTE or jointType==p.JOINT_PRISMATIC]
LineID = visualizeEEframe(robotId, len(MovableJointIdx))

# Load a sphere to visualize coordinate frame obtained by DH method
obj_color = [1, 0, 0, 1]
visualShapeId = p.createVisualShape(shapeType=p.GEOM_SPHERE, 
                                    radius=0.01, 
                                    rgbaColor=obj_color)
ballId = p.createMultiBody(baseMass=0,
                           baseInertialFramePosition =[0, 0, 0],
                           baseVisualShapeIndex=visualShapeId,
                           basePosition=[0, 0, 1])
visualizeDHframe(ballId)

[3, 4, 5]

# Forward Kinematics

### DH notation

In [9]:
#TODO: Define functions for FK by DH
def updateDHparams(dh_parameters, joint_variables, jointTypes):
    pass

def getRelativeTransformation(dh_parameter):
    pass

def FK_DH(dh_parameters):
    pass

### Functions required to get end-effector's transformation

In [10]:
def getTfromRp(R, pos):
    T = np.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = pos
    return T

def getEETransformation(robotId, n=7):
    """
    Get end effector's homogeneous transformation w.r.t. world frame.

    :param robotId: robot's id returned when loading urdf.
    :param n: index of robot's link that we want to observe.
    :return: Robot link's position and orientation in the form of transformation matrix.
    """
    linkState = p.getLinkState(robotId, n-1) 
    pos = linkState[4]
    R = p.getMatrixFromQuaternion(linkState[5])
    R = np.array(R).reshape(3,3)
    return getTfromRp(R, pos)
    # phi, theta, psi = p.getEulerFromQuaternion(linkState[5]) # XYZ Euler angle
    # cphi = np.cos(phi)
    # sphi = np.sin(phi)
    # ctheta = np.cos(theta)
    # stheta = np.sin(theta)
    # cpsi = np.cos(psi)
    # spsi = np.sin(psi)
    # R = [[ctheta * cpsi, -ctheta * spsi, stheta],
    #      [sphi * stheta * cpsi + cphi * spsi, -sphi * stheta * spsi + cphi * cpsi, -sphi * ctheta],
    #      [-cphi * stheta * cpsi + sphi * spsi, cphi * stheta * spsi + sphi * cpsi, cphi * ctheta]]
    
def rotation_matrix_to_quaternion(R):
    """
    Convert a rotation matrix into a quaternion.

    :param R: A 3x3 rotation matrix.
    :return: A quaternion in the form of [x, y, z, w].
    """
    m00, m01, m02 = R[0, 0], R[0, 1], R[0, 2]
    m10, m11, m12 = R[1, 0], R[1, 1], R[1, 2]
    m20, m21, m22 = R[2, 0], R[2, 1], R[2, 2]
    
    trace = m00 + m11 + m22

    if trace > 0:
        s = 0.5 / np.sqrt(trace + 1.0)
        w = 0.25 / s
        x = (m21 - m12) * s
        y = (m02 - m20) * s
        z = (m10 - m01) * s
    elif m00 > m11 and m00 > m22:
        s = 2.0 * np.sqrt(1.0 + m00 - m11 - m22)
        w = (m21 - m12) / s
        x = 0.25 * s
        y = (m01 + m10) / s
        z = (m02 + m20) / s
    elif m11 > m22:
        s = 2.0 * np.sqrt(1.0 + m11 - m00 - m22)
        w = (m02 - m20) / s
        x = (m01 + m10) / s
        y = 0.25 * s
        z = (m12 + m21) / s
    else:
        s = 2.0 * np.sqrt(1.0 + m22 - m00 - m11)
        w = (m10 - m01) / s
        x = (m02 + m20) / s
        y = (m12 + m21) / s
        z = 0.25 * s

    return np.array([x, y, z, w])

# Test

### Set target joint variables

In [12]:
#TODO: Set arbitrary joint variables
q = np.array([0,0,0,0,0,0]) * np.pi / 180

### Get desired transformation using DH method

In [13]:
#TODO: Define Indy7 v2 DH parameters


#TODO: Get EE's Transformation matrix using FK by DH
T_dh = FK_DH(dh_parameters)

# visualization part, do not touch
R_dh = T_dh[0:3, 0:3]
pos_dh = T_dh[0:3, 3].T
quat_dh = rotation_matrix_to_quaternion(R_dh)
p.resetBasePositionAndOrientation(ballId, pos_dh, quat_dh)

### Apply joint variables to robot and get current transformation

In [14]:
#TODO: Apply joint variables to robot and get EE's Transformation matrix

# p.setJointMotorControlArray()
# sleep(0.5)

In [15]:
T_simul = getEETransformation(robotId, numJoints)

print(np.matmul(np.linalg.inv(T_dh), T_simul)) # This should be eye(4)


[[-0.644 0.477 0.598 0.042]
 [-0.369 0.491 -0.789 -0.179]
 [-0.670 -0.729 -0.140 -0.426]
 [0.000 0.000 0.000 1.000]]


In [16]:
#p.disconnect()